In [1]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [100]:
QQQ = pd.read_csv('QQQ_consol_20050101_to_202006021.csv')
QQQ.drop('Date', axis =1, inplace = True)

In [101]:
# predict loop test
def pred(input):
    return np.sum(np.array(input[-60:])/60)

forecast = 5
for i in range(forecast):
    
    y_pred = pred(QQQ)
    new_row ={'Close':y_pred}
    QQQ = QQQ.append(new_row, ignore_index = True,)

In [102]:
QQQ[-8:]

,Close,RSI,12 EMA,26 EMA
3865,243.619995,63.019095,237.840681,232.405092
3866,244.279999,57.916519,238.729806,233.235825
3867,244.240005,0.000000,239.583682,234.053912
3868,708.813971,NaN,NaN,NaN
3869,NaN,NaN,NaN,NaN
3870,NaN,NaN,NaN,NaN
3871,NaN,NaN,NaN,NaN
3872,NaN,NaN,NaN,NaN


In [78]:
QQQ.tail()

,Close
Date,
2020-06-15,238.750000
2020-06-16,242.850006
2020-06-17,243.619995
2020-06-18,244.279999
2020-06-19,244.240005


In [90]:
QQQ.tail()

,Close
3933,230.113101
3934,230.120653
3935,230.138664
3936,230.136142
3937,230.138578
